In [1]:
import numpy as np
import pandas as pd
import json
import plotly.express as px

/var/folders/k0/b16qfl8d0wb9sg1p820yhndm0000gn/T/ipykernel_29926/3763631997.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# config = {
#     'person1': {
#         "age_starting": 24,
#         "age_retirement": 60,
#         "age_sim_end": 100,
#         "number_of_runs": 10000,
#         "amount_starting": 0,
#         "contribution_monthly": 3500,
#         "retirement_drawdown_monthly": 10000,
#         "inflation_mean": 0.05,
#         "inflation_std":  0.01,
#         "growth_mean": 0.10,
#         "growth_std": 0.010,
#     }
# }
# config_json = json.dumps(config, indent=2)
# with open('config.json', 'w') as outfile:
#     outfile.write(config_json)

In [3]:
config = json.load(open('config.json'))['person1']

In [4]:
config['growth_std'] = config['growth_std']/np.sqrt(12)  # fact sheets shows annulaised standard devivation...

In [5]:
periods = (config['age_sim_end'] - config['age_starting'])*12 + 1
drawdown_start_period = (config['age_retirement'] - config['age_starting'])*12 + 1

In [6]:
size = (periods, config['number_of_runs'])

In [7]:
inflation = np.random.normal(config['inflation_mean'], config['inflation_std'], size=size)
growth = np.random.normal(config['growth_mean'], config['growth_std'], size=size)
delta_growth = (growth - inflation)/12

In [8]:
scenarios = np.ones(size)

In [9]:
scenarios[0,:] = config['amount_starting']
for p in range(periods)[1:]:
    if drawdown_start_period > p:
        scenarios[p,:] = (scenarios[p-1,:] + config['contribution_monthly']) * (1+delta_growth[p,:])
    else:
        scenarios[p,:] = (scenarios[p-1,:] - config['retirement_drawdown_monthly']) * (1+delta_growth[p,:])
        
scenarios[scenarios < 0] = 0
scenarios.shape

(733, 10000)

In [10]:
percentiles = [1,10,25,50,75,90,99]
scenario_stats = np.percentile(scenarios, percentiles, axis=1)
scenario_stats.shape

(7, 733)

In [11]:
l = [f"{y}-{m}" for y in range(config['age_starting'], config['age_sim_end']) for m in range(1,13)]

In [12]:
plot_data = pd.DataFrame(scenario_stats.T, columns = [f"{p}" for p in percentiles])
plot_data.head()
plot_data['age'] = [f"{y:04}-{m:02}" for y in range(config['age_starting'], config['age_sim_end']+1) for m in range(1,13)][0:len(plot_data)]
plot_data = pd.melt(plot_data, id_vars=['age'], var_name='percentile', value_name='value')

In [13]:
fig = px.line(plot_data, x='age', y='value', color='percentile')
fig.write_html("result.html")